In [ ]:
# !pip install "transformers==4.28.0"
# !pip install "tokenizers>=0.19,<0.23"
# !pip install git+https://github.com/thunlp/OpenPrompt.git


In [ ]:
# !sed -i 's/transformers.generation_utils/transformers.generation/' /usr/local/lib/python3.12/dist-packages/openprompt/pipeline_base.py

In [ ]:
# # Fix all AdamW imports in openprompt
# !grep -rl "from transformers import  AdamW" /usr/local/lib/python3.12/dist-packages/openprompt | xargs sed -i 's/from transformers import  AdamW, get_linear_schedule_with_warmup/from torch.optim import AdamW\nfrom transformers import get_linear_schedule_with_warmup/'


In [1]:
# Set device
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# ==============================
# Imports
# ==============================
import pandas as pd
import torch
import random
import numpy as np
from collections import Counter
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
from openprompt.prompts import ManualTemplate, ManualVerbalizer
from openprompt.prompts import SoftVerbalizer
from openprompt.prompts import AutomaticVerbalizer
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt import PromptForClassification, PromptDataLoader
from torch.optim import AdamW
from sklearn.metrics import classification_report

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\requests\__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(
C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\transformers\generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [3]:
# ==============================
# Set random seeds for reproducibility
# ==============================
seed = 20
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
# ==============================
# Step 1: Training data (3-shot / few-shot)
# ==============================
train_dataset = [
    #Politics
    InputExample(guid=0, text_a="بلوچستان ۾ ايراني سرزمين تان ٻيهر حملو، پاڪ فوج جا 4 جوان شهيد", label=0),
    InputExample(guid=1, text_a="سپريم ڪورٽ ۾ قرآن پاڪ جي حافظ کي 20 واڌو نمبر ڏيڻ بابت پاڻمرادي نوٽيس ۾ جسٽس قاضي فائز عيسيٰ جي سربراهي ۾ 3 رڪني خاص بئنچ فيصلو جاري ڪيو", label=0),
    InputExample(guid=2, text_a="واضح رهي ته ڪيرول ئي نه پر ڪيترين ئي عورتن اڳوڻي آمريڪي صدر تي جنسي ڏاڍائي ۾ ملوث هجڻ جا الزام لڳايا آهن.", label=0),
    InputExample(guid=3, text_a="پيادل قافلي جي ڀرپور آجيان ڪري مٿن گلن جي ورکا ڪئي وئي.", label=0),

    #Sports
    InputExample(guid=4, text_a="نيپال جي ڪرڪيٽ ميچ ڊي ايل ايس ميٿڊ تحت ون ڊي ۾ 9 وڪيٽن تي يو اي اي جي ٽيم کي شڪست ڏئي ڇڏي آهي", label=1),
    InputExample(guid=5, text_a="هوڏانهن ٻئي سيمي فائنل ۾ وري آريانا سيبالينڪا يونان جي ماريا سڪاري کي شڪست ڏئي فائنل ۾ جڳهه ٺاهي آهي، هاڻ فائنل ۾ آريانا سيبالينڪا ۽ ايلانا ريباڪنا آمهون سامهون ٿينديون.", label=1),
    InputExample(guid=6, text_a="جڏهن تھ سريلنڪا ويمن جي ٽيم 19 اوور ۾ رڳو 3 وڪيٽن جي نقصان تي هدف پورو ڪري ڪاميابي ماڻي ورتي، جن ۾ چماري اتاپٽو 33، هرشيٿا سماراوڪرما 29، ڪويشا ڊلهاري 20 رنسن سان نمايان رهيون", label=1),
    InputExample(guid=7, text_a="ڪرسٽيانو رونالڊو 2023ع دوران 38 گول ڪري سڀني کان اڳيان نڪري ويو", label=1),

    #Crime
    InputExample(guid=8, text_a="پر گڏيل قومن جي هڪ بيان ۾ چيو ويو آهي ته موت جي سزا ماڻيندڙن جو اصل انگ ان کان وڌيڪ آهي.", label=2),
    InputExample(guid=9, text_a="هڪ عورت کي سندس ڌيءَ سميت ڏوهاري اغوا ڪري ويا.", label=2),
    InputExample(guid=10, text_a="پرڏيهي ميڊيا موجب ان ڳالهه جو انڪشاف هڪ ايراني نائب وزير يونس پناهي ڪيو آهي، جنهن چيو آهي ته ايران ۾ اسڪول جي ڇوڪرين کي زهر ڏنو ويو", label=2),
    InputExample(guid=11, text_a="ڳڙهي خدابخش کوسو ۾ نينگري جي قتل جو الزام.", label=2),
]


In [5]:
# ==============================
# Define Classes
# ==============================
classes = ["Politics", "Sports", "Crime"]
label_map = {"Politics": 0, "Sports": 1, "Crime":2}

# # Step 1: Use load_plm with 'roberta' to get the correct WrapperClass
_, _, _, WrapperClass = load_plm("roberta", "roberta-base")  # Just to get the wrapper

# # Step 2: Manually load XLM-RoBERTa model/tokenizer
model_name = "xlm-roberta-large"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
plm = XLMRobertaForMaskedLM.from_pretrained(model_name)

# ==============================
# Load Pretrained Language Model (mBERT)
# ==============================
# plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-multilingual-cased")

# ==============================
# Define Prompt Template (Manual)
# ==============================
template = ManualTemplate(
    # text = '{"placeholder":"text_a"} هذه الجملة {"mask"}.',
    text = '{"placeholder":"text_a"} هي جملا آهي {"mask"}.',
    tokenizer=tokenizer,
)

# ==============================
# Define Verbalizer (Manual)
# ==============================
verbalizer = ManualVerbalizer(
    classes=classes,
    label_words = {
        "Politics": ["سياست", "حڪومت"],       # (Politics, Government)
        "Sports": ["کيل", "ميچ"],             # (Game, Match)
        "Crime": ["جُرم", "قتل"]            # (Crime, Murder)
    },
    tokenizer=tokenizer,
)


C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
# # Apply PromptDA → augment training data
# augmented_train_dataset = []
# for example in train_dataset:
#     class_name = classes[example.label]
#     class_index = classes.index(class_name)
#     label_words = verbalizer.label_words[class_index]
#     for lw in label_words:
#         augmented_train_dataset.append(InputExample(
#             guid=f"{example.guid}_{lw}",
#             text_a=example.text_a,
#             label=example.label
#         ))


# ==============================
# Create Prompt Model
# ==============================
prompt_model = PromptForClassification(
    template=template,
    plm=plm,
    verbalizer=verbalizer
)


# # DataLoader for training (use augmented data) for when using promptda
# train_loader = PromptDataLoader(
#     dataset=augmented_train_dataset,
#     tokenizer=tokenizer,
#     template=template,
#     tokenizer_wrapper_class=WrapperClass,
#     max_seq_length=128,
#     batch_size=4,
#     shuffle=True,
# )



# # ==============================
# # DataLoader for Training
# # ==============================
train_loader = PromptDataLoader(
    dataset=train_dataset,
    tokenizer=tokenizer,
    template=template,
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=4,
    shuffle=True
)

tokenizing: 12it [00:00, 2985.09it/s]


In [7]:
# ==============================
# Fine-Tuning the Prompt Model
# ==============================
prompt_model.train()
optimizer = AdamW(prompt_model.parameters(), lr=1e-5)

for epoch in range(5):
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        logits = prompt_model(batch)
        loss = torch.nn.CrossEntropyLoss()(logits, batch['label'])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1} Loss: {total_loss:.4f}")


Epoch 1 Loss: 5.4685
Epoch 2 Loss: 2.0234
Epoch 3 Loss: 1.7611
Epoch 4 Loss: 0.7012
Epoch 5 Loss: 0.4418


In [8]:
# ==============================
# Load Evaluation Dataset
# ==============================
df = pd.read_csv(r"C:\Users\stdFurqan\Desktop\Sindhi (politics)\split_50.csv")
eval_dataset = [
    InputExample(guid=i, text_a=row['News Headlines'], label=label_map[row['Aspect Category']])
    for i, row in df.iterrows()
]

eval_loader = PromptDataLoader(
    dataset=eval_dataset,
    tokenizer=tokenizer,
    template=template,
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=8,
    shuffle=False
)

tokenizing: 8277it [00:01, 4890.15it/s]


In [9]:
# ==============================
# Evaluate Model and Collect Predicted MASK Words for Correct Predictions
# ==============================
prompt_model.eval()

correct_predicted_words = {
    "Politics": [],
    "Sports": [],
    "Crime": []}

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in eval_loader:
        logits = prompt_model(batch)
        preds = torch.argmax(logits, dim=-1)

        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(batch['label'].cpu().tolist())

        input_ids = batch['input_ids']
        mask_token_id = tokenizer.mask_token_id

        for i in range(len(preds)):
            true_label = batch['label'][i].item()
            pred_label = preds[i].item()

            if true_label == pred_label:
                # Find MASK token index
                mask_index = (input_ids[i] == mask_token_id).nonzero(as_tuple=True)[0].item()

                # Prepare input for PLM forward to get token logits
                inputs = {
                    "input_ids": input_ids[i].unsqueeze(0),
                    "attention_mask": batch['attention_mask'][i].unsqueeze(0)
                }
                if 'token_type_ids' in batch:
                    inputs["token_type_ids"] = batch['token_type_ids'][i].unsqueeze(0)

                # outputs = plm(**inputs)
                # token_logits = outputs.logits[0, mask_index]
                outputs = prompt_model.plm(**inputs)
                token_logits = outputs.logits[0, mask_index]   # logits for vocab at MASK position

                predicted_token_id = torch.argmax(token_logits).item()
                predicted_word = tokenizer.convert_ids_to_tokens(predicted_token_id).lstrip("##")

                class_name = classes[true_label]
                correct_predicted_words[class_name].append(predicted_word)



In [10]:
# ==============================
# Print Classification Report and Predicted Words Summary
# ==============================
print("\n📊 Fine-tuned Classification Report:")
print(classification_report(all_labels, all_preds, target_names=classes))

print("\nPredicted MASK words for correct predictions by class:")

for cls in classes:
    words = correct_predicted_words[cls]
    word_counts = Counter(words)
    print(f"\nClass '{cls}':")
    print(f"Total correct predictions: {len(words)}")
    print("Most common predicted mask words:")
    for w, cnt in word_counts.most_common(10):
        print(f"  {w} : {cnt}")


📊 Fine-tuned Classification Report:
              precision    recall  f1-score   support

    Politics       0.63      0.99      0.77      3802
      Sports       0.95      0.74      0.83      2075
       Crime       0.97      0.29      0.45      2400

    accuracy                           0.72      8277
   macro avg       0.85      0.67      0.68      8277
weighted avg       0.81      0.72      0.69      8277


Predicted MASK words for correct predictions by class:

Class 'Politics':
Total correct predictions: 3754
Most common predicted mask words:
  ▁سياسي : 721
  . : 661
  ▁سياست : 407
  ▁غلط : 192
  ▁قانون : 167
  ▁ڪرپشن : 159
  ▁حڪومت : 113
  ▁صحيح : 94
  ▁اهم : 83
  ▁حقيقت : 81

Class 'Sports':
Total correct predictions: 1531
Most common predicted mask words:
  . : 1130
  ▁هڪ : 130
  : : 74
  ▁بهترين : 61
  ▁صحيح : 25
  ▁0 : 24
  ▁عظيم : 8
  ، : 7
  ▁دلچسپ : 6
  ▁شاندار : 4

Class 'Crime':
Total correct predictions: 706
Most common predicted mask words:
  . : 284
  ▁قتل : 158
